In [10]:
import pandas as pd
# import pdpipe as pdp
from pulp import *

In [11]:
full_players = pd.read_csv('full_players.csv', index_col=0).reset_index(drop=True)
# full_players = full_players[::10]

full_players['Score'] = pd.to_numeric(full_players['Score'].replace('-',0))
schedule = pd.read_csv('2021_schedule', index_col=0) 

players_2021 = full_players[full_players['Year']==2021][['Name','Price','Team 1', 'Position']]
player_prev = full_players[full_players['Name'].isin(players_2021['Name'].unique())]
player_prev = player_prev.groupby(['Name','Team 2']).mean()['Score']

players_schedule_2021 = players_2021.merge(schedule, on=['Team 1'])
players_schedule_score_2021 = players_schedule_2021.merge(player_prev, on = ['Name','Team 2'])

P = players_schedule_score_2021
P['id'] = "R:"+P['Round'].astype(str)+"_P:"+P['Name']
P = P[P['Round']<4]

In [12]:
# full_players = pd.read_csv('full_players.csv', index_col=0).reset_index(drop=True)
# len(full_players['Name'].unique())
# # len(full_players[full_players['Year']==2021]['Name'].unique())
# # len(P['Name'].unique())

In [13]:
player_contraints = {}
prob = LpProblem("aflProblem", LpMaximize)

overall_score = LpVariable('OverallScore',0)
player_contraints = LpVariable.dicts("Player Contraints", P['id'].unique(), 0, 1, cat='Binary')
# prob += lpSum([player_contraints[p_id[1]]*score[1] for p_id,score in zip(P['id'].iteritems(),P['Score'].iteritems())]), "Total score is maximized"

In [14]:
P_prev_round = P.copy()
P_prev_round['Round_prev'] = P_prev_round['Round'] - 1

P_transfers = P[['id','Position','Round','Name']].merge(P_prev_round[['id','Position','Round_prev','Name']],left_on=['Position','Round'],right_on=['Position','Round_prev'], suffixes = ('_prev','_next'))

P_transfers['Transfer'] = P_transfers['id_prev']+'-'+P_transfers['id_next']
P_transfers=P_transfers[['id_prev',	'Round',	'Name_prev',	'id_next',	'Round_prev',	'Name_next',	'Transfer']].drop_duplicates()

transfer_contraints = LpVariable.dicts("Transfer contraints", P_transfers['Transfer'], 0, 1, cat='Binary')

# map rounds end to itermediary
for prev_player,trans in P_transfers.groupby(['id_prev'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[prev_player], f"Previous player equals transfer intermediatary for {prev_player} equals {trans}"

# map intermediary to next round
for next_player,trans in P_transfers.groupby(['id_next'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in trans]) == player_contraints[next_player], f"Next player equals transfer intermediatary for {next_player} equals {trans}"

for round, transfers in P_transfers[P_transfers['Name_prev'] != P_transfers['Name_next']][['Round','Transfer']].drop_duplicates().groupby(['Round'])['Transfer'].apply(list).iteritems():
    prob += lpSum([transfer_contraints[t_id] for t_id in transfers]) <= 2, f"Round: {round}, has less than or equal to 4 transfers"

# # END TRANFERS

In [15]:
# START money contraint
money = 13 * 10**6
for round,player in P[['id','Price','Round']].drop_duplicates().groupby(['Round'])['id'].apply(list).iteritems():
    prob += lpSum([player_contraints[p_id]*P[P['id']==p_id]['Price'].values[0] for p_id in player]) <= money, f"Round: {round}, has less than ${money}"
# # END money contraint

In [16]:
#START Contrain each player to a single position per round
P_Position_Casting = P.copy()
P_Position_Casting['pl_pos_id'] = (P['id']+'_'+P['Position'])
P_Position_Casting = P_Position_Casting[['id','pl_pos_id','Round','Position']].drop_duplicates()

Players_Position_Casting = LpVariable.dicts("Players_Position_Casting", P_Position_Casting['pl_pos_id'], 0, 1, cat='Binary')

Players_Position_Casting_Series = P_Position_Casting[['id','pl_pos_id','Round']].drop_duplicates().groupby(['id','Round'])['pl_pos_id'].apply(list).iteritems()

for (p_id,round), pl_pos_ids in Players_Position_Casting_Series:
    player_possible_positions = [Players_Position_Casting[pl_pos_id] for pl_pos_id  in pl_pos_ids] 
    prob += lpSum(player_possible_positions) <= player_contraints[p_id], f"Must have only have equal to {p_id} (0 or 1) of {player_possible_positions} positions in round {round}"
  #END Contrain each player to a single position per round

In [17]:
# # START max players from each position
allowed_holds_per_position = {'DE': 8, "MI" : 10, 'RU' : 3, 'FO':9}
# allowed_holds_per_position = {'DE': 1, "MI" : 0, 'RU' : 0, 'FO':0}
for (position,round), pl_pos_ids in P_Position_Casting[['pl_pos_id','Position','Round']].drop_duplicates().groupby(['Position', 'Round'])['pl_pos_id'].apply(list).iteritems():
    prob += lpSum([Players_Position_Casting[pl_pos_id] for pl_pos_id in pl_pos_ids]) == allowed_holds_per_position[position], f"Position: {position}, has less than {allowed_holds_per_position[position]} in round {round}"
# END max players from each position

In [9]:
try:
    solver = getSolver('COIN_CMD', timeLimit=4000, msg=True,gapRel = 0.2)
    prob.solve(solver)
except:
    print('USING INBUILT SOLVER!!  \n')
    solver = getSolver('PULP_CBC_CMD')
    prob.solve(solver)

USING INBUILT SOLVER!!  



NameError: name 'prob' is not defined

In [ ]:
P.groupby(['Position','Round']).count()

In [ ]:
prob.status

In [ ]:
results = []
for name, player_position in Players_Position_Casting.items():
    if player_position.value() != 0:
        results.append([name,1])
Players_selected = pd.DataFrame(results, columns=['pl_pos_id', 'is_selected'])
P_Position_Casting = P_Position_Casting.merge(Players_selected, on = ['pl_pos_id'])
P_Position_Casting.to_csv('solution.csv')
print(P_Position_Casting.groupby(['Round','Position']).sum()['is_selected'])